Common Voice Dataset:
Use this dataset to train the model creating the fake voices (negative label)
Large datasets, start with delta segment 15.0

TIMIT:
Real speakers speaking sentences (positive label)
Used to generate the generation model


Step 1:
TIMIT 
speech synthesis - Tacotron
find pre trained model and finetune



In [1]:
import torchaudio
from speechbrain.pretrained import Tacotron2
from speechbrain.pretrained import HIFIGAN

# Intialize TTS (tacotron2) and Vocoder (HiFIGAN)
tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder")

# Running the TTS
mel_output, mel_length, alignment = tacotron2.encode_text("Mary had a little lamb")

# Running Vocoder (spectrogram-to-waveform)
waveforms = hifi_gan.decode_batch(mel_output)

# Save the waverform
torchaudio.save('example_TTS.wav',waveforms.squeeze(1), 22050)

h:\Lighthouse Labs\Apziva\Project6\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
h:\Lighthouse Labs\Apziva\Project6\.venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
h:\Lighthouse Labs\Apziva\Project6\.venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dis

In [2]:
from speechbrain.pretrained import Tacotron2
tacotron2 = Tacotron2.from_hparams(source="speechbrain/TTS_Tacotron2", savedir="tmpdir")
items = [
       "A quick brown fox jumped over the lazy dog",
       "How much wood would a woodchuck chuck?",
       "Never odd or even"
     ]
mel_outputs, mel_lengths, alignments = tacotron2.encode_batch(items)

# Running Vocoder (spectrogram-to-waveform)
for i, output in enumerate(mel_outputs):
    waveform = hifi_gan.decode_batch(output)
    torchaudio.save(f'example_TTS_{i}.wav',waveform.squeeze(1), 22050)

# the audio files (1,2) contain some beeping

In [5]:
import re

In [6]:
# now to read text files containing the text to be converted to speech
# in timit/lisa/data/timit/raw/TIMIT/DOC/PROMPTS.TXT

path = '../data/timit/lisa/data/timit/raw/TIMIT/DOC/PROMPTS.TXT'

lines = []

# Open file and read contents
with open(path, 'r') as f:
    lines = [re.sub(r'\([^)]*\)', '', line.strip()) for line in f.readlines() if not line.startswith(';')]

print(lines)

['She had your dark suit in greasy wash water all year. ', "Don't ask me to carry an oily rag like that. ", 'This was easy for us. ', 'Jane may earn more money by working hard. ', 'She is thinner than I am. ', 'Bright sunshine shimmers on the ocean. ', 'Nothing is as offensive as innocence. ', 'Why yell or worry over silly items? ', 'Where were you while we were away? ', "Are your grades higher or lower than Nancy's? ", 'He will allow a rare lie. ', 'Will Robin wear a yellow lily? ', 'Swing your arm as high as you can. ', "Before Thursday's exam, review every formula. ", 'The museum hires musicians every evening. ', 'A roll of wire lay near the wall. ', 'Carl lives in a lively home. ', "Alimony harms a divorced man's wealth. ", 'Aluminum silverware can often be flimsy. ', 'She wore warm, fleecy, woolen overalls. ', 'Alfalfa is healthy for you. ', 'When all else fails, use force. ', 'Those musicians harmonize marvelously. ', 'Although always alone, we survive. ', 'Only lawyers love mill

In [41]:
# remove extra whitespaces
def remove_extra_whitespace(text):
    # Replace multiple whitespaces after a period with a single space
    cleaned_text = re.sub(r'\.\s+', '.', text)
    return cleaned_text

cleaned_lines = []
for line in lines:
    cleaned_lines.append(remove_extra_whitespace(line))

In [42]:
cleaned_lines

['Those answers will be straightforward if you think them through carefully first.',
 'Each untimely income loss coincided with the breakdown of a heating system part.',
 'X-ray films of the vertebral column showed progression of the demineralization.',
 'With those paintings of big constructions crashing down, he felt he could stop.',
 'With the spring rains the flow rose rapidly due to infiltration in open sewers.',
 'What a discussion can ensue when the title of this type of song is in question.',
 'We seemed to be witnessing the population explosion right in our own backyards.',
 'We knew that it was, as reassurance, the ironic fruit of a deeply moral nature.',
 "We flew in rickety planes so overloaded that we wondered why they didn't crash.",
 'Unit prices for state vehicles are invariably lower than to the general public.',
 'To prepare mustard cream, blend mustard with enough water to make a thin paste.',
 'To date, the one meat showing favorable results at sterilization doses i

In [46]:
tacotron2 = Tacotron2.from_hparams(source="speechbrain/TTS_Tacotron2", savedir="tmpdir")
items = cleaned_lines

# Sort items by length in decreasing order
sorted_items = sorted(items, key=len, reverse=True)

# Call encode_batch with sorted items
mel_outputs, mel_lengths, alignments = tacotron2.encode_batch(sorted_items)

# Running Vocoder (spectrogram-to-waveform)
for i, (output, length) in enumerate(zip(mel_outputs, mel_lengths)):
    waveform = hifi_gan.decode_batch(output[:, :length])
    torchaudio.save(f'../output/prompt_{i}.wav', waveform.squeeze(0), 22050)

AssertionError: input lengths must be sorted in decreasing order

In [48]:
for i, item in enumerate(sorted_items):
    mel_output, mel_length, alignment = tacotron2.encode_text(item)
    # Running Vocoder (spectrogram-to-waveform)
    waveforms = hifi_gan.decode_batch(mel_output)

    # Save the waverform
    torchaudio.save(f'../output/prompt_{i}.wav', waveforms.squeeze(1), 22050)

KeyboardInterrupt: 

In [2]:
# for common voice dataset, try to choose similar characteristics. sentence length, voice, 
# to run it in parallel, needs to have same text content

In [3]:
import pandas as pd

C:\Users\jared\AppData\Local\Temp\ipykernel_12672\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df = pd.read_csv('../data/Common Voice/cv-corpus-15.0-delta-2023-09-08-en/cv-corpus-15.0-delta-2023-09-08/en/validated.tsv', sep= '\t')

In [5]:
df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,0432ddcb259937fdea308281cf359701843b7e45cb068b...,common_voice_en_38400274.mp3,The bow was towed to the Marghera shipyard whe...,4,0,NaN,NaN,"polish,Slavic,Foreign,Easter European",NaN,en,NaN
1,074d49c2d50d4d49f4a380888b24d824b09f4bc046edc8...,common_voice_en_38229408.mp3,The black-and-white video depicts a sad Shakir...,2,0,NaN,NaN,NaN,NaN,en,NaN
2,0c3cfe9c1348466c4c418eba79a794ae6b85f242c234ed...,common_voice_en_38029302.mp3,Also at Allen Park is the relocated Tom Collin...,2,0,NaN,NaN,NaN,NaN,en,NaN
3,0d4af81e1ef834b86153f6f3e1f86bc2450488cbcdc3bf...,common_voice_en_38096117.mp3,They subsequently formed an oppositional group...,2,0,twenties,female,"India and South Asia (India, Pakistan, Sri Lanka)",NaN,en,NaN
4,0dbd9404fe8c3d057dfc6091eb7a8b2f1218d9814d38c7...,common_voice_en_38057509.mp3,"Today, corporate-run prisons hold eight percen...",2,0,NaN,NaN,"United States English,Californian",NaN,en,NaN


In [6]:
df['accents'].unique()

array(['polish,Slavic,Foreign,Easter European', nan,
       'India and South Asia (India, Pakistan, Sri Lanka)',
       'United States English,Californian',
       'India and South Asia (India, Pakistan, Sri Lanka),United States English',
       'United States English,England English,Transatlantic English',
       'England English,Very British ', 'United States English',
       'India and South Asia (India, Pakistan, Sri Lanka),Bangladeshi English',
       'United States English,midwestern united states',
       'United States English,Midwestern',
       'Australian English,Canadian English', 'indonesia',
       'Canadian English', 'Kenyan ', 'Australian English',
       'England English',
       'United States English,England English,India and South Asia (India, Pakistan, Sri Lanka)',
       'United States English,England English', 'Deformato',
       'England English,Northern England', 'German English',
       'England English,India and South Asia (India, Pakistan, Sri Lanka)',
     

In [7]:
# filter by female and with english accent similar to TIMIT TTS
filtered_df = df[(df['gender'] == 'female') & (df['accents'].str.contains('English'))]

In [8]:
filtered_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
46,85df70ed0a409cdfa99faed45ba84b8443c28541aa1212...,common_voice_en_38291751.mp3,"A splendid meteor fell, spreading dismay throu...",2,0,fifties,female,United States English,NaN,en,NaN
105,6b9290e8ad5c3f330e9608588a89f47872e4a709870f0e...,common_voice_en_38287702.mp3,Women must remain chaste while planting pumpki...,2,0,sixties,female,United States English,NaN,en,NaN
106,6b9290e8ad5c3f330e9608588a89f47872e4a709870f0e...,common_voice_en_38287730.mp3,They may now leave the tent by the ordinary do...,2,0,sixties,female,United States English,NaN,en,NaN
123,f138d75a743543591fb23c61bbb2cfc4a204866aa791fd...,common_voice_en_38359630.mp3,"Autumn starts out warm but becomes cooler, wet...",2,0,twenties,female,United States English,NaN,en,NaN
124,f138d75a743543591fb23c61bbb2cfc4a204866aa791fd...,common_voice_en_38359634.mp3,Not all Birmingham Dribblers are Victorian ant...,2,0,twenties,female,United States English,NaN,en,NaN


In [9]:
filtered_df.iloc[0]['path']

'common_voice_en_38291751.mp3'

In [10]:
voice_dataset = pd.DataFrame(columns=['path', 'label'])

In [13]:
voice_dataset['path'] = filtered_df['path']

In [14]:
voice_dataset['label'] = 'no'

In [15]:
voice_dataset.head()

,path,label
46,common_voice_en_38291751.mp3,no
105,common_voice_en_38287702.mp3,no
106,common_voice_en_38287730.mp3,no
123,common_voice_en_38359630.mp3,no
124,common_voice_en_38359634.mp3,no


In [16]:
voice_dataset.shape

(101, 2)

In [21]:
voice_dataset.iloc[0]['path']

'common_voice_en_38291751.mp3'

In [18]:
# moving files
import shutil

In [29]:
common_voice_root = '../data/Common Voice/cv-corpus-15.0-delta-2023-09-08-en/cv-corpus-15.0-delta-2023-09-08/en/clips'

for i in range(voice_dataset.shape[0]):
    shutil.copyfile(f'{common_voice_root}/{voice_dataset.iloc[i]["path"]}', f'../dataset/no/no_{i+1}.wav')

In [34]:
# no = not fake, yes = fake
import os
# create yes dataset
for i, file in enumerate(os.listdir('../output/')):
    shutil.copyfile(f'../output/{file}', f'../dataset/yes/yes_{i+1}.wav')

In [ ]:
# move the dataset to docker

In [12]:
# wav file to vector 